In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [52]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [53]:
# Data Preprocessing
# Drop unnecessary columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [54]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [55]:
print(data['Geography'].value_counts())
print(data['Gender'].value_counts())

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64
Gender
Male      5457
Female    4543
Name: count, dtype: int64


In [56]:
# encode categorical variables
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6460,582,France,1,29,4,0.00,2,0,0,156153.27,0
3699,749,France,1,37,10,185063.70,2,1,1,134526.87,0
7311,707,France,1,37,1,0.00,2,0,1,6035.51,0
8259,619,Spain,1,38,3,96143.47,1,0,0,98994.92,0
863,717,France,1,53,6,0.00,2,0,1,97614.87,0


In [57]:
# one-hot encoding for 'Geography'
from sklearn.preprocessing import OneHotEncoder
ohe_geo=OneHotEncoder()
geo_encoded=ohe_geo.fit_transform(data[['Geography']])
geo_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [58]:
ohe_geo.get_feature_names_out(['Geography'])


array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [59]:
geo_encoded_df=pd.DataFrame(geo_encoded.toarray(),columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [60]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [61]:
data['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [62]:
# save the label encoder and scaler
with open('label_encoder_gender','wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('ohe_geo','wb') as file:
    pickle.dump(ohe_geo, file)
    

In [63]:
# splitting the data into dep and independent variables
X=data.drop('Exited',axis=1)
y=data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaling the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)



In [64]:
# save the scaler
with open('scaler','wb') as file:
    pickle.dump(scaler, file)


In [65]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


## ANN implementation ##

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [67]:

cnt=0
for x in data.columns:
    cnt+=1
print(cnt)
(X_train.shape[1])

13


12

In [68]:
# building the model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), # hl1
    Dense(32,activation='relu') ,# hl2
    Dense(1,activation='sigmoid') # output layer
])

In [69]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                832       
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [70]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [71]:
# compiling the model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])


In [72]:
# set up the tensortboard
log_dir="log/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [73]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [74]:
# set up the early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10, restore_best_weights=True)

In [75]:
# training the model
history=model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 2s 4ms/step - loss: 0.4079 - accuracy: 0.8273 - val_loss: 0.3478 - val_accuracy: 0.8585
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3572 - accuracy: 0.8562 - val_loss: 0.3466 - val_accuracy: 0.8610
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3458 - accuracy: 0.8561 - val_loss: 0.3526 - val_accuracy: 0.8585
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3474 - accuracy: 0.8577 - val_loss: 0.3420 - val_accuracy: 0.8600
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3425 - accuracy: 0.8599 - val_loss: 0.3465 - val_accuracy: 0.8520
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3394 - accuracy: 0.8612 - val_loss: 0.3357 - val_accuracy: 0.8600
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3357 - accuracy: 0.8622 - val_loss: 0.3424 - val_accuracy: 0.8590

In [76]:
model.save('model.h5')


c:\Users\nandi\Desktop\ann-classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [77]:
# load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [78]:
!taskkill /PID 4236 /F


ERROR: The process "4236" not found.


In [79]:
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 4236), started 1:16:26 ago. (Use '!kill 4236' to kill it.)

In [80]:
# load the pickle file
